# housekeeping

In [2]:
! pwd

/Users/domolm/Library/CloudStorage/GoogleDrive-domokomod5@gmail.com/My Drive/Learning/rakus/CI coding practice/Python


In [ ]:
import pandas as pd
import matplotlib as plt

# 3.8